In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\Asus\\Desktop\\kidney_Disease_Classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Asus\\Desktop\\kidney_Disease_Classifier'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/sharmantima1010/Kidney_Disease_Classifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sharmantima1010"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b6e96687cb64b794bcb2f2b3b6a1f3651e45009f"

In [6]:
pip show tensorflow


Name: tensorflowNote: you may need to restart the kernel to use updated packages.

Version: 2.20.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\asus\anaconda3\envs\kidney\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google_pasta, grpcio, h5py, keras, libclang, ml_dtypes, numpy, opt_einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing_extensions, wrapt
Required-by: 


In [7]:
import tensorflow as tf

In [8]:
model= tf.keras.models.load_model("artifacts/training/model.keras")

In [9]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int
    

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self)->EvaluationConfig:
        eval_config=EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/sharmantima1010/Kidney_Disease_Classifier.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE

        )


    

        return eval_config

In [12]:
pip install mlflow==2.17.0


Note: you may need to restart the kernel to use updated packages.


In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config=config
    
    def _valid_generator(self):
        
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self._valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        model=tf.keras.models.load_model(path)
        return model
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self._valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(
            path=Path("scores.json"),
            data=scores
        )
    

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model,
                    "model",
                    registered_model_name="KidneyDiseaseClassifierModel"
                )
            else:
                mlflow.keras.log_model(
                    self.model,
                    "model"
                )
            
        
    

In [15]:
try:
    config= ConfigurationManager()
    eval_config=config.get_evaluation_config()
    
    evaluation= Evaluation(eval_config)
    evaluation.evaluation()
    
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-10-31 19:58:38,105: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-31 19:58:38,110: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-31 19:58:38,112: INFO: common: created directory at: artifacts_root]
Found 3732 images belonging to 4 classes.


c:\Users\Asus\anaconda3\envs\kidney\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


234/234 ━━━━━━━━━━━━━━━━━━━━ 157s 661ms/step - accuracy: 0.9046 - auc: 0.9692 - loss: 0.6042 - precision: 0.9097 - recall: 0.8966
[2025-10-31 20:01:15,883: INFO: common: json file saved at: scores.json]


2025/10/31 20:01:18 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/10/31 20:01:26 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Asus\AppData\Local\Temp\tmprpvh30fz\model, flavor: keras). Fall back to return ['keras==3.11.3']. Set logging level to DEBUG to see the full traceback. 
2025/10/31 20:01:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'KidneyDiseaseClassifierModel' already exists. Creating a new version of this model...
2025/10/31 20:04:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: KidneyDiseaseClassifierModel, version 3
Created version '3' of model 'KidneyDiseaseClassifierModel'.
2025/10/31 20:04:59 INFO mlflow.tracking._trackin

In [1]:
pip install dagshub

  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
   ---------------------------------------- 0.0/14.1 MB ? eta -:--:--
   --- ------------------------------------ 1.3/14.1 MB 9.6 MB/s eta 0:00:02
   -------- ------------------------------- 3.1/14.1 MB 8.0 MB/s eta 0:00:02
   -------------- ------------------------- 5.0/14.1 MB 8.2 MB/s eta 0:00:02
   -------------------- ------------------- 7.3/14.1 MB 9.2 MB/s eta 0:00:01
   ------------

In [3]:
import dagshub
dagshub.init(repo_owner='sharmantima1010', repo_name='Kidney_Disease_Classifier', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1afb8208-6fdc-4cd3-94f8-2a938097cd55&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=f22796ea8035706a4487b82934e1764839b6243aab52258046c8a48534523433




Accessing as sharmantima1010

Initialized MLflow to track repo "sharmantima1010/Kidney_Disease_Classifier"

Repository sharmantima1010/Kidney_Disease_Classifier initialized!

2025/10/29 14:01:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run mercurial-turtle-925 at: https://dagshub.com/sharmantima1010/Kidney_Disease_Classifier.mlflow/#/experiments/0/runs/ee8db2962cac4efd8c48ba03710b44f0.
2025/10/29 14:01:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sharmantima1010/Kidney_Disease_Classifier.mlflow/#/experiments/0.
